In [ ]:
from oandapyV20 import API
from oandapyV20.contrib.factories import InstrumentsCandlesFactory
import oandapyV20.endpoints.instruments as instruments
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
import configparser

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice

In [ ]:
config = configparser.ConfigParser()
config.read('../config/oanda_conf.ini')

access_token = config['LIVE']['TOKEN']

api = API(access_token=access_token, environment="live")

In [ ]:
# 日時による取得
def fetch_data_by_date(_from, _to, granularity='M5', instrument='USD_JPY'):
    data = []
    # APIに渡すパラメーターの設定
    params = {
        'from': _from,
        'to': _to,
        'granularity':granularity
    }
    # APIへ過去為替レートをリクエスト
    for r in InstrumentsCandlesFactory(instrument=instrument, params=params):
        api.request(r)
        for raw in r.response['candles']:
            data.append([raw['time'], raw['volume'], raw['mid']['o'], raw['mid']['h'], raw['mid']['l'], raw['mid']['c']])

    df = pd.DataFrame(data)
    df.columns = ['Time', 'Volume', 'Open', 'High', 'Low', 'Close']
    df = df.set_index('Time')
    df.index = pd.to_datetime(df.index, format='%Y%m%d %H:%M:%S')
    
    today = datetime.today().strftime("%m%d")
    f_name = '{}-{}-{}-{}.csv'.format(instrument, granularity, today, len(df))
    
    df.to_csv(f_name)
    return df, f_name

In [ ]:
def get_data(x_df, y_df, window_len=10, train_rate = 0.8):
    mms = MinMaxScaler()
    x_df = mms.fit_transform(x_df)
    y_df = mms.fit_transform(y_df)
    x, y = [], []
    for i in range(len(df)-window_len):
        x.append(x_df[i:i+window_len])
        y.append(y_df[i+window_len])
    
    train_size = int(train_rate * (len(df)-window_len))
    x_train, y_train, x_test, y_test = np.array(x[:train_size]), np.array(y[:train_size]), np.array(x[train_size:]), np.array(y[train_size:])
    return x_train, y_train, x_test, y_test

In [ ]:
def build_model(inputs, output_size, neurons=300, activ_func="linear",
                dropout=0.5, loss="mae", optimizer="adam"):
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(None, inputs.shape[1], inputs.shape[2]), dropout=dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
 
    model.compile(loss=loss, optimizer=optimizer)
    return model

In [ ]:
def create_model(x_train, y_train, x_test, y_test):
    model = Sequential()
    model.add(LSTM({{choice([256, 512])}}, input_shape=(x_train.shape[1], x_train.shape[2]), dropout={{uniform(0, 1)}}))
    model.add(Dense(units=y_train.shape[1]), activation="linear")
 
    model.compile(loss="mae", optimizer="adam")
    early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=0)
    model.fit(x_train, y_train,
              batch_size=100,
              epochs=10,
              verbose=1,
              validation_split=0.2,
              callbacks=[early_stopping])

    val_loss, val_acc = model.evaluate(x_test, y_test, verbose=0)
    return {'loss': -val_acc, 'status': STATUS_OK, 'model': model}

In [ ]:
today = datetime.today()
one_years_ago = today - relativedelta(years=1)
one_month_ago = today - relativedelta(months=1)

iso_format = '%Y-%m-%dT%H:%M:%SZ'

granularity='M5'
instrument='USD_JPY'

_from = one_years_ago.strftime(iso_format)
_to = today.strftime(iso_format)

df, f_name = fetch_data_by_date(_from, _to)
df

In [ ]:
f_name = 'USD_JPY-M5-0621-74286.csv'
df = pd.read_csv(f_name, encoding="utf_8", index_col=0)

In [ ]:
x_df = df[['Open', 'High', 'Low', 'Close']]
y_df = df[['Close']]

In [ ]:
x_train, y_train, x_test, y_test = get_data(x_df, y_df, window_len=10)
for data in [x_train, y_train, x_test, y_test]:
    print(data.shape)

In [ ]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=get_data,
                                      algo=tpe.suggest,
                                      max_evals=100,
                                      trials=Trials())

print(best_model.summary())
print(best_run)

_, _, x_test, y_test = get_data()
val_loss, val_acc = best_model.evaluate(x_test, y_test)
print("val_loss: ", val_loss)
print("val_acc: ", val_acc)

In [ ]:
output_size = y_train.shape[1]
model = build_model(x_train, output_size=output_size, neurons=600)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=0)
history = model.fit(x_train, y_train, batch_size=600, epochs=10, validation_split=0.2, callbacks=[early_stopping])
val_loss, val_acc = model.evaluate(x_test, y_test, verbose=0)

In [ ]:
# MAEをプロットしてみよう
fig, ax1 = plt.subplots(1,1)
 
ax1.plot(history.epoch, history.history['loss'])
ax1.set_title('TrainingError')
ax1.set_ylabel('Model Loss',fontsize=12)
ax1.set_xlabel('# Epochs',fontsize=12)
plt.show()

In [ ]:
pred_data = model.predict(x_train)
plt.plot(y_train.reshape(pred_data.shape[0]), label='train')
plt.plot(pred_data, label='pred')
plt.legend(loc='upper left')
plt.show()

In [ ]:
pred_data = model.predict(x_test)
plt.plot(y_test.reshape(pred_data.shape[0]), label='test')
plt.plot(pred_data, label='pred')
plt.legend(loc='upper left')
plt.show()